In [ ]:
!pip install optuna

In [ ]:
!pip install optuna_integration

In [ ]:
# ignore warnings
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger('lightgbm').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## **Read dataset**

In [ ]:

project_path = '/content/drive/My Drive/Code Documentation Project/Issue Classification/'

In [ ]:
from numpy import loadtxt
import pandas as pd
from tensorflow.keras.utils import to_categorical  # convert to one-hot-encoding
from sklearn import preprocessing


issue_df = pd.read_csv(project_path+'literature_comments_dataset.csv')
#issue_df = pd.read_csv(project_path+'Dataset2.csv')


label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(issue_df['Code'])
issue_df['Code (Original)'] = issue_df['Code']
issue_df['Code'] = label_encoder.transform(issue_df['Code'])

#number_of_class
class_id_map_df = issue_df.drop_duplicates(subset=['Code'])
total_class=len(class_id_map_df)
encoded_label_list = class_id_map_df['Code'].to_list()
original_label_list = class_id_map_df['Code (Original)'].to_list()
class_id_map = dict(zip(encoded_label_list, original_label_list))

categ = ['aa','begauth','has_code','first_turn','last_turn']
le = preprocessing.LabelEncoder()
issue_df[categ] = issue_df[categ].apply(le.fit_transform)

print(len(issue_df))


4330


In [ ]:
issue_df['Code (Original)'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import optuna_integration.lightgbm
import lightgbm as lgb
import numpy as np
from lightgbm import early_stopping
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna

Fold=10
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
for fold in range(1,Fold+1):
    features=['len','tloc','cloc','tpos1','tpos2','clen','tlen','ppau','npau','aa','begauth','has_code','first_turn','last_turn']

    train_df=pd.read_csv(project_path+'/KFold/Literature/train_fold'+str(fold)+'.csv')
    X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['Text Content'])
    X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    print(X_train_tfidf.shape)

    val_df=pd.read_csv(project_path+'/KFold/Literature/val_fold'+str(fold)+'.csv')
    X_val_tfidf   = tfidf_vectorizer.transform(val_df['Text Content'])
    X_val_tfidf_df   = pd.DataFrame(X_val_tfidf.toarray(),   columns=tfidf_vectorizer.get_feature_names_out())
    print(X_val_tfidf.shape)

    test_df=pd.read_csv(project_path+'/KFold/Literature/test_fold'+str(fold)+'.csv')
    X_test_tfidf  = tfidf_vectorizer.transform(test_df['Text Content'])
    X_test_tfidf_df  = pd.DataFrame(X_test_tfidf.toarray(),  columns=tfidf_vectorizer.get_feature_names_out())
    print(X_test_tfidf.shape)

    X_train = train_df[features] #'Full Length',
    X_train = pd.concat([train_df[features], X_train_tfidf_df], axis=1)
    print(X_train.shape)
    y_train = train_df['Code']

    X_val= val_df[features] #'Full Length',
    X_val = pd.concat([val_df[features], X_val_tfidf_df], axis=1)
    print(X_val.shape)
    y_val = val_df['Code']

    X_test = test_df[features] #'Full Length',
    X_test = pd.concat([test_df[features], X_test_tfidf_df], axis=1)
    print(X_test.shape)
    print(X_val.shape)
    y_test = test_df['Code']


    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)


    params = {
        "objective": "multiclass",
        "num_classes":13,
        "class_weight":'balanced',
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }

    model = optuna_integration.lightgbm.train(
        params,
        dtrain,
        valid_sets=[dtrain, dval],
        callbacks=[early_stopping(10)],
    )

    best_params = model.params

    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))



    clf = lgb.LGBMClassifier(**best_params)

    X_train_val = pd.concat([X_train, X_val], axis=0)
    y_train_val = pd.concat([y_train, y_val], axis=0)
    clf.fit(X_train_val, y_train_val)

    y_pred=clf.predict(X_test)

    label_list = []
    for i in range(13):
      label_list.append(class_id_map[i])

    true_labels = []
    pred_labels = []


    for e in y_test:
      true_labels.append(class_id_map[e])


    for e in y_pred:
      pred_labels.append(class_id_map[e])

    #print(y_pred)
    pred_df=pd.read_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv')
    pred_df['LGBM']=y_pred
    pred_df.to_csv(project_path+'/Ensemble Data/Literature/fold'+str(fold)+'.csv', index=False)
    print(classification_report(true_labels,pred_labels))

    report= classification_report(true_labels,pred_labels, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df = report_df.round(2)
    report_df.to_csv(project_path+'classification report/Literature/lgbm_tfidf/lgbm_fold'+str(fold)+'.csv', index = False)
    #report_df.to_csv(str(fold)+'.csv', index = False)
    #break

In [ ]:
sup=[0 for i in range(16)]
pr=[0 for i in range(16)]
rc=[0 for i in range(16)]
f1=[0 for i in range(16)]

TOTAL_FOLD=10
for fold in range(1,TOTAL_FOLD+1):
  report_df=pd.read_csv(project_path+'classification report/Literature/lgbm_tfidf/lgbm_fold'+str(fold)+'.csv')
  #print(report_df)
  for i in range(0,16):
    s=report_df['support'][i]
    sup[i]=sup[i]+report_df['support'][i]
    pr[i]=pr[i]+(s*report_df['precision'][i])
    rc[i]=rc[i]+(s*report_df['recall'][i])
    f1[i]=f1[i]+(s*report_df['f1-score'][i])

for i in range(0,16):
  pr[i]=pr[i]/sup[i]
  rc[i]=rc[i]/sup[i]
  f1[i]=f1[i]/sup[i]
  sup[i]=int(sup[i])

pr = [ round(i, 2) for i in pr ]
rc = [ round(i, 2) for i in rc ]
f1 = [ round(i, 2) for i in f1 ]

report_df = pd.DataFrame({'pr':pr,'rc':rc,'f1':f1,'sup':sup})
print(report_df)
report_df.to_csv(project_path+'classification report/Literature/lgbm_tfidf/total.csv')

